In [1]:
# Install deps
!pip install -q sentence-transformers urlextract tldextract beautifulsoup4 xgboost

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re, string, math
from bs4 import BeautifulSoup
import tldextract
from urlextract import URLExtract
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

# --- 1. Load datasets ---
phishX_ai = pd.read_csv("/content/drive/MyDrive/llm-generated/emails.csv", on_bad_lines='skip')
legit_ai = pd.read_csv("/content/drive/MyDrive/llm-generated/legit.csv")
phish_ai = pd.read_csv("/content/drive/MyDrive/llm-generated/phishing.csv", on_bad_lines='skip')
legit_human = pd.read_csv("/content/drive/MyDrive/llm-generated/H_legit.csv")
phish_human = pd.read_csv("/content/drive/MyDrive/llm-generated/H_phishing.csv", on_bad_lines='skip')

# --- 2. Add labels ---
phishX_ai["label"] = 1
phishX_ai["source"] = "AI"

legit_ai["label"] = 0
legit_ai["source"] = "AI"

phish_ai["label"] = 1
phish_ai["source"] = "AI"

legit_human["label"] = 0
legit_human["source"] = "human"

phish_human["label"] = 1
phish_human["source"] = "human"

# --- 3. Combine ---
df = pd.concat([ phishX_ai,legit_ai, phish_ai,legit_human, phish_human], ignore_index=True)

# --- 4. Deduplicate (exact matches) ---
before = len(df)
df = df.drop_duplicates()
print(f"Removed {before - len(df)} exact duplicates")

# --- 5. Drop rows without labels ---
df = df.dropna(subset=["label"])

# --- 6. Train/test split (stratified on label, but preserve both sources in both sets) ---
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

print("\n===== Dataset Summary =====")
print(f"Total samples: {len(df)}")
print(f"Train size: {len(train_df)}, Test size: {len(test_df)}")
print("\nTrain distribution:\n", train_df["label"].value_counts())
print("\nTest distribution:\n", test_df["label"].value_counts())
print("\nSources in train:", train_df["source"].value_counts().to_dict())
print("Sources in test:", test_df["source"].value_counts().to_dict())


Removed 928 exact duplicates

===== Dataset Summary =====
Total samples: 2690
Train size: 2152, Test size: 538

Train distribution:
 label
0    1598
1     554
Name: count, dtype: int64

Test distribution:
 label
0    400
1    138
Name: count, dtype: int64

Sources in train: {'human': 1186, 'AI': 966}
Sources in test: {'human': 318, 'AI': 220}


In [3]:
save_path = '/content/drive/MyDrive/llm-generated/phishing_dataset_combined1.csv'
df.to_csv(save_path, index=False)
print(f"\nDataset saved to: {save_path}")



Dataset saved to: /content/drive/MyDrive/llm-generated/phishing_dataset_combined1.csv


In [4]:
# Load your combined dataset
df = pd.read_csv("/content/drive/MyDrive/llm-generated/phishing_dataset_combined1.csv")
#df.head(50)
print(f"Total samples: {len(df)}")

Total samples: 2690


In [5]:
print(df.head())
#print(df['label'].value_counts())  # Check class balance
df.head(50)
print(f"Total samples: {len(df)}")

                                                text  label source sender  \
0  Urgent: Verify your account now to avoid suspe...      1     AI    NaN   
1  Action required: Your mailbox storage is almos...      1     AI    NaN   
2  Security alert: We detected suspicious sign-in...      1     AI    NaN   
3  Your payment failed. Please update billing inf...      1     AI    NaN   
4  We noticed unauthorized activity. Verify your ...      1     AI    NaN   

  receiver date subject body urls  
0      NaN  NaN     NaN  NaN  NaN  
1      NaN  NaN     NaN  NaN  NaN  
2      NaN  NaN     NaN  NaN  NaN  
3      NaN  NaN     NaN  NaN  NaN  
4      NaN  NaN     NaN  NaN  NaN  
Total samples: 2690


In [6]:
#Column mapping (auto-detect common names)
# Try to robustly locate text columns
def find_col(candidates):
    for c in candidates:
        if c in df.columns: return c
    return None

COL_SUBJ = find_col(["subject","Subject","email_subject","title"])
COL_TEXT = find_col(["body_text","Body","text","email_body","content","message"])
COL_HTML = find_col(["body_html","html","email_html"])

if COL_TEXT is None and COL_HTML is None:
    raise ValueError("No text-like columns found. Add a 'body_text' or 'body_html' column.")

print("Using columns:", {"subject": COL_SUBJ, "text": COL_TEXT, "html": COL_HTML})


Using columns: {'subject': 'subject', 'text': 'text', 'html': None}


In [7]:
import requests
#Preprocessing & feature extraction
from urlextract import URLExtract # Import URLExtract
import string # Import string
import re # Import re
import numpy as np # Import numpy
from bs4 import BeautifulSoup # Import BeautifulSoup
import tldextract # Import tldextract

url_extractor = URLExtract()
SUS_WORDS = {
    "verify","verification","urgent","password","account","locked","suspend","invoice",
    "wire","transfer","bank","gift","card","login","update","security","confirm","winner",
    "click","payment","overdue","reset","unlock","limited","expire","otp","2fa"
}
PUNCT = set(string.punctuation)

def strip_html(html):
    if not isinstance(html, str) or not html.strip():
        return ""
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup(["script","style"]): tag.decompose()
    text = soup.get_text(separator=" ")
    return re.sub(r"\s+", " ", text).strip()

def join_text(row):
    # Ensure inputs are strings, handling potential non-string values (like NaN)
    subj = str(row[COL_SUBJ]) if COL_SUBJ and pd.notna(row[COL_SUBJ]) else ""
    body_txt = str(row[COL_TEXT]) if COL_TEXT and pd.notna(row[COL_TEXT]) else ""
    body_html = str(row[COL_HTML]) if COL_HTML and pd.notna(row[COL_HTML]) else ""

    # prefer longer between text and stripped HTML
    stripped = strip_html(body_html)
    body = body_txt if len(body_txt) >= len(stripped) else stripped
    return (subj + "\n\n" + body).strip()

df["joined_text"] = df.apply(join_text, axis=1)

def normalize(text):
    if not isinstance(text, str): text = ""
    # Lowercase & collapse whitespace
    text = text.replace("\u200b", " ").replace("\ufeff", " ")
    text = re.sub(r"\s+", " ", text.strip()).lower()
    return text

df["clean_text"] = df["joined_text"].apply(normalize)

def extract_urls_raw(text):
    try:
        return list(set(url_extractor.find_urls(text, only_unique=True)))
    except:
        return []


# Common URL shorteners
SHORTENER_DOMAINS = {
    "bit.ly", "tinyurl.com", "goo.gl", "t.co", "ow.ly", "buff.ly",
    "rebrand.ly", "is.gd", "cutt.ly", "shorte.st", "trib.al", "lnkd.in"
}

def get_redirect_chain_length(url, timeout=5):
    """
    Follow redirects and return chain length.
    """
    try:
        r = requests.get(url, timeout=timeout, allow_redirects=True)
        return len(r.history)
    except Exception:
        return 0

def classic_features(row):
    if not isinstance(row, (pd.Series, dict)):
        raise TypeError("Input to classic_features must be a pandas Series or dictionary.")

    text = row.get("joined_text", "")
    clean = row.get("clean_text", "")
    urls = extract_urls_raw(text)

    url_lengths = [len(u) for u in urls] or [0]
    domains = []
    has_ip = 0
    has_shortener = 0
    max_redirects = 0
    has_redirects = 0   # 🔹 new flag

    for u in urls:
        ext = tldextract.extract(u)
        host = ".".join([p for p in [ext.subdomain, ext.domain, ext.suffix] if p])
        domains.append(host)

        # IP in host
        if re.match(r"^\d{1,3}(\.\d{1,3}){3}$", host):
            has_ip = 1

        # Shortener detection
        if host in SHORTENER_DOMAINS:
            has_shortener = 1

        # Redirect chain
        redirects = get_redirect_chain_length(u)
        if redirects > 0:
            has_redirects = 1
        if redirects > max_redirects:
            max_redirects = redirects

    words = re.findall(r"\b[a-z0-9]+\b", clean)
    susp_count = sum(w in SUS_WORDS for w in words)
    char_len = len(text)
    word_len = len(words)
    avg_word_len = (sum(map(len, words))/word_len) if word_len else 0.0
    digit_ratio = sum(c.isdigit() for c in text)/char_len if char_len else 0.0
    uppercase_ratio = sum(c.isupper() for c in text)/char_len if char_len else 0.0
    exclaim_count = text.count("!")

    spf = int(row.get("spf_pass", 0)) if pd.notna(row.get("spf_pass")) and str(row.get("spf_pass")).isdigit() else 0
    dkim = int(row.get("dkim_pass", 0)) if pd.notna(row.get("dkim_pass")) and str(row.get("dkim_pass")).isdigit() else 0
    dmarc = int(row.get("dmarc_pass", 0)) if pd.notna(row.get("dmarc_pass")) and str(row.get("dmarc_pass")).isdigit() else 0

    features_series = pd.Series({
        "char_len": char_len,
        "word_len": word_len,
        "avg_word_len": avg_word_len,
        "digit_ratio": digit_ratio,
        "uppercase_ratio": uppercase_ratio,
        "exclaim_count": exclaim_count,
        "susp_count": susp_count,
        "num_urls": len(urls),
        "num_domains": len(set(domains)),
        "max_url_len": max(url_lengths),
        "mean_url_len": float(np.mean(url_lengths)),
        "has_ip": has_ip,
        "has_shortener": has_shortener,   #  NEW
        "has_redirects": has_redirects,   #  NEW
        "max_redirects": max_redirects,   #  NEW
        "spf_pass": spf,
        "dkim_pass": dkim,
        "dmarc_pass": dmarc,
        "has_attachments": int(pd.notna(row.get("has_attachments", 0)) and row.get("has_attachments", 0) != 0),
        "num_attachments": int(pd.notna(row.get("num_attachments", 0)) and row.get("num_attachments", 0) != 0),
    })
    return pd.DataFrame([features_series])

# Apply classic_features and concatenate the resulting DataFrames
classic_df = pd.concat(df.apply(classic_features, axis=1).tolist(), ignore_index=True)
classic_df.head()


,char_len,word_len,avg_word_len,digit_ratio,uppercase_ratio,exclaim_count,susp_count,num_urls,num_domains,max_url_len,mean_url_len,has_ip,has_shortener,has_redirects,max_redirects,spf_pass,dkim_pass,dmarc_pass,has_attachments,num_attachments
0,121.0,20.0,4.800000,0.000000,0.024793,0.0,7.0,1.0,1.0,28.0,28.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,115.0,19.0,4.842105,0.000000,0.026087,0.0,0.0,1.0,1.0,30.0,30.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,127.0,19.0,5.473684,0.000000,0.023622,0.0,5.0,1.0,1.0,30.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,131.0,18.0,5.944444,0.000000,0.015267,0.0,3.0,1.0,1.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,112.0,17.0,5.352941,0.017857,0.035714,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# BERT embeddings (Sentence-Transformers)
from sentence_transformers import SentenceTransformer
bert = SentenceTransformer("all-MiniLM-L6-v2")  # 384-dim, fast, solid quality
emb = bert.encode(df["clean_text"].tolist(), batch_size=64, show_progress_bar=True, normalize_embeddings=True)
emb_df = pd.DataFrame(emb)
print(emb_df.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/43 [00:00<?, ?it/s]

(2690, 384)


In [9]:
# RNN branch (BiLSTM on tokenized sequences) - NO LEAKAGE
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split
import pandas as pd

# Drop rows with missing labels before splitting
#df_cleaned = df.dropna(subset=['label']).copy()
y = df["label"].astype(int).values

# Split first (so tokenizer doesn’t see val data)
X_train, X_val, y_train, y_val = train_test_split(
    df["clean_text"], df["label"].astype(int), # Use df_cleaned
    test_size=0.2,
    random_state=42,
    stratify=df["label"] # Stratify on df_cleaned
)

# Tokenize only on training data
MAX_VOCAB = 30000
MAX_LEN = 256
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<unk>")
tokenizer.fit_on_texts(X_train)  # 👈 only training texts

# Convert to sequences
X_seq_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=MAX_LEN, padding="post", truncating="post")
X_seq_val   = pad_sequences(tokenizer.texts_to_sequences(X_val),   maxlen=MAX_LEN, padding="post", truncating="post")

# Manual class weights (tune these if needed)
class_weights = {
    0: 1.0,   # legit
    1: 5.0    # phishing
}
print("Manual Class Weights:", class_weights)

# Build BiLSTM model
EMB_DIM = 128
lstm_model = models.Sequential([
    layers.Embedding(input_dim=MAX_VOCAB, output_dim=EMB_DIM, input_length=MAX_LEN),
    layers.SpatialDropout1D(0.2),   # 👈 helps reduce overfitting at embedding level
    layers.Bidirectional(layers.LSTM(32, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
    layers.GlobalMaxPooling1D(),
    layers.Dense(32, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

# Compile
lstm_model.compile(
    optimizer=optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["AUC","Precision","Recall"]
)
lstm_model.summary()

# Callbacks
es = callbacks.EarlyStopping(
    patience=2, restore_best_weights=True,
    monitor="val_AUC", mode="max"
)
rlr = callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=1, verbose=1
)

# Train with class weights
hist = lstm_model.fit(
    X_seq_train, y_train,
    validation_data=(X_seq_val, y_val),
    epochs=10,
    batch_size=128,
    callbacks=[es, rlr],
    verbose=1,
    class_weight=class_weights   # 👈 here
)

# Get probabilities for later fusion (on all data if needed)
# Use df_cleaned for getting probabilities as well, to match the trained model
all_seq = pad_sequences(tokenizer.texts_to_sequences(df["clean_text"]), maxlen=MAX_LEN)
rnn_probs = lstm_model.predict(all_seq, batch_size=512).ravel()
rnn_df = pd.DataFrame({"rnn_prob": rnn_probs})

Manual Class Weights: {0: 1.0, 1: 5.0}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - AUC: 0.6126 - Precision: 0.3066 - Recall: 0.7157 - loss: 1.3818 - val_AUC: 0.9381 - val_Precision: 0.2565 - val_Recall: 1.0000 - val_loss: 0.7611 - learning_rate: 0.0010
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 19s 934ms/step - AUC: 0.7906 - Precision: 0.3265 - Recall: 0.9942 - loss: 1.1715 - val_AUC: 0.9540 - val_Precision: 0.4355 - val_Recall: 0.9783 - val_loss: 0.5518 - learning_rate: 0.0010
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 925ms/step - AUC: 0.9239 - Precision: 0.5139 - Recall: 0.9686 - loss: 0.7857 - val_AUC: 0.9801 - val_Precision: 0.7360 - val_Recall: 0.9493 - val_loss: 0.2480 - learning_rate: 0.0010
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 958ms/step - AUC: 0.9839 - Precision: 0.7879 - Recall: 0.9743 - loss: 0.3939 - val_AUC: 0.9917 - val_Precision: 0.8221 - val_Recall: 0.9710 - val_loss: 0.1433 - learning_rate: 0.0010
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 969ms/step - AUC: 0.9908 - Precision: 0.8636 - Recall: 0.9832

In [10]:
print(df['label'].value_counts(normalize=True))


label
0    0.742751
1    0.257249
Name: proportion, dtype: float64


In [11]:
unique_labels = df['label'].unique()
print("Unique labels:", unique_labels)


Unique labels: [1 0]


In [12]:
import numpy as np
# y_train, y_val = ...  # however you split
print("Train distribution:", np.bincount(y_train))
print("Val distribution:", np.bincount(y_val))

Train distribution: [1598  554]
Val distribution: [400 138]


In [13]:
# Fuse features & Isolation Forest (unsupervised anomaly)
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# Scale classic features
scaler = StandardScaler()
classic_scaled = scaler.fit_transform(classic_df.values)
classic_scaled_df = pd.DataFrame(classic_scaled, columns=classic_df.columns)

# Fused vector for anomaly detection: [BERT | classic]
Z = np.hstack([emb, classic_scaled])
mask_legit = (y == 0)
Z_legit = Z[mask_legit]

iforest = IsolationForest(
    n_estimators=400, contamination=0.05, random_state=42, n_jobs=-1
).fit(Z_legit)

raw_if = -iforest.score_samples(Z)  # larger => more anomalous
# Normalize 5–95th percentile on legit for a 0..1 score
lo, hi = np.percentile(-iforest.score_samples(Z_legit), [5, 95])
if_anom = np.clip((raw_if - lo) / (hi - lo + 1e-9), 0, 1)
anom_df = pd.DataFrame({"if_anom": if_anom})


In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np

# Dictionary of feature blocks
feature_blocks = {
    "emb": emb,
    "classic": classic_scaled,
    "rnn": rnn_df.values,
    "anom": anom_df.values,
}

print("\n===== Leakage Diagnostic =====")
for name, feat in feature_blocks.items():
    print(f"\n--- Testing {name} ---")
    X_train, X_test, y_train, y_test = train_test_split(
        feat, y, test_size=0.2, random_state=42, stratify=y
    )

    xgb = XGBClassifier(
        n_estimators=100, max_depth=3, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        eval_metric="auc", tree_method="hist", random_state=42
    )
    xgb.fit(X_train, y_train)
    probs = xgb.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, probs)
    print(f"{name} ROC-AUC: {auc_score:.4f}")



===== Leakage Diagnostic =====

--- Testing emb ---
emb ROC-AUC: 0.9977

--- Testing classic ---
classic ROC-AUC: 0.9439

--- Testing rnn ---
rnn ROC-AUC: 0.9887

--- Testing anom ---
anom ROC-AUC: 0.7258


In [15]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from collections import Counter
import numpy as np

# Build meta-features (403 cols)
X_meta = np.hstack([emb, classic_scaled, rnn_df.values, anom_df.values])

X_train, X_test, y_train, y_test = train_test_split(
    X_meta, y, test_size=0.2, random_state=42, stratify=y
)

# Retrain properly
xgb = XGBClassifier(
    n_estimators=500, max_depth=6, learning_rate=0.05,
    subsample=0.9, colsample_bytree=0.9,
    eval_metric="logloss", tree_method="hist", random_state=42
)
xgb.fit(X_train, y_train)

print("XGB expects:", xgb.n_features_in_)   # should now print 403
#xgb.fit(X_train, y_train)

probs = xgb.predict_proba(X_test)[:,1]
preds = (probs >= 0.5).astype(int)

print(classification_report(y_test, preds, digits=4))
print("ROC-AUC:", roc_auc_score(y_test, probs))

prec, rec, thr = precision_recall_curve(y_test, probs)
print("PR-AUC:", auc(rec, prec))

from collections import Counter
print("Confusion matrix:\n", confusion_matrix(y_test, preds))
print("Pred balance:", Counter(preds))

# Save fresh model to artifacts
#joblib.dump(xgb, "artifacts/meta_xgb.joblib")

XGB expects: 406
              precision    recall  f1-score   support

           0     0.9875    0.9900    0.9888       400
           1     0.9708    0.9638    0.9673       138

    accuracy                         0.9833       538
   macro avg     0.9792    0.9769    0.9780       538
weighted avg     0.9832    0.9833    0.9833       538

ROC-AUC: 0.9980978260869565
PR-AUC: 0.9947758966017325
Confusion matrix:
 [[396   4]
 [  5 133]]
Pred balance: Counter({np.int64(0): 401, np.int64(1): 137})


In [16]:
# inference function
# Persist artifacts (tokenizer, scalers, iforest, models)
import joblib
import os, json
os.makedirs("artifacts", exist_ok=True)

joblib.dump(scaler, "artifacts/classic_scaler.joblib")
joblib.dump(iforest, "artifacts/isolation_forest.joblib")
joblib.dump((lo, hi), "artifacts/iforest_bounds.joblib")
joblib.dump(xgb, "artifacts/meta_xgb.joblib")
lstm_model.save("artifacts/rnn_bilstm.keras")
bert.save("artifacts/sbert")  # sentence-transformers saves folder
joblib.dump(tokenizer, "artifacts/tokenizer.joblib")
classic_df.columns.to_series().to_json("artifacts/classic_feature_names.json")

print("Artifacts saved.")
def score_email(subject="", body_text="", body_html="",
                spf_pass=0, dkim_pass=0, dmarc_pass=0,
                has_attachments=0, num_attachments=0,
                THRESHOLD=0.5 ):
    # Construct single-row DataFrame
    row = {
        COL_SUBJ: subject if COL_SUBJ else "",
        COL_TEXT: body_text if COL_TEXT else "",
        COL_HTML: body_html if COL_HTML else "",
        "spf_pass": spf_pass,
        "dkim_pass": dkim_pass,
        "dmarc_pass": dmarc_pass,
        "has_attachments": has_attachments,
        "num_attachments": num_attachments
    }
    row_df = pd.DataFrame([row])

    # Clean text
    row_df["joined_text"] = row_df.apply(join_text, axis=1)
    row_df["clean_text"] = row_df["joined_text"].apply(normalize)

    # --- Classic features ---
    feats_df = classic_features(row_df.iloc[0])  # returns DataFrame
    classic_vec = scaler.transform(feats_df.values).reshape(1, -1)  # (1, 17)

    # --- BERT embedding ---
    emb_vec = bert.encode([row_df["clean_text"].iloc[0]], normalize_embeddings=True)
    emb_vec = emb_vec.reshape(1, -1)  # (1, 384)

    # --- RNN probability ---
    seq = tokenizer.texts_to_sequences([row_df["clean_text"].iloc[0]])
    seq = pad_sequences(seq, maxlen=MAX_LEN, padding="post", truncating="post")
    rnn_prob = float(lstm_model.predict(seq, verbose=0).ravel()[0])
    rnn_arr = np.array([[rnn_prob]])  # (1, 1)

    # --- Anomaly score ---
    Z_one = np.hstack([emb_vec, classic_vec])  # (1, 401)
    raw = -iforest.score_samples(Z_one)[0]
    lo_, hi_ = lo, hi
    if_score = float(np.clip((raw - lo_) / (hi_ - lo_ + 1e-9), 0, 1))
    if_arr = np.array([[if_score]])  # (1, 1)

    # --- Meta features (final 403-dim vector) ---
    x_meta = np.hstack([emb_vec, classic_vec, rnn_arr, if_arr])  # (1, 403)

    # --- Prediction ---
    p = float(xgb.predict_proba(x_meta)[0, 1])
    label = int(p >= THRESHOLD)

    # Risk bands
    if p < 0.3:
        risk = "low"
    elif p < 0.7:
        risk = "medium"
    else:
        risk = "high"

    return {
        "prob": p,
        "label": label,
        "risk": risk,
        "rnn_prob": rnn_prob,
        "if_anom": if_score
    }
# Quick test
print(score_email(subject="Urgent: verify your account",
                  body_text="Please confirm your password at http://login-secure-update.com",
                  ))




Artifacts saved.
{'prob': 0.9996808767318726, 'label': 1, 'risk': 'high', 'rnn_prob': 0.9927936792373657, 'if_anom': 0.38093074915115793}


In [17]:
!zip -r /content/drive/My\ Drive/my_folder.zip /content/artifacts

  adding: content/artifacts/ (stored 0%)
  adding: content/artifacts/sbert/ (stored 0%)
  adding: content/artifacts/sbert/1_Pooling/ (stored 0%)
  adding: content/artifacts/sbert/1_Pooling/config.json (deflated 59%)
  adding: content/artifacts/sbert/config.json (deflated 47%)
  adding: content/artifacts/sbert/tokenizer_config.json (deflated 73%)
  adding: content/artifacts/sbert/model.safetensors (deflated 9%)
  adding: content/artifacts/sbert/sentence_bert_config.json (deflated 9%)
  adding: content/artifacts/sbert/special_tokens_map.json (deflated 80%)
  adding: content/artifacts/sbert/config_sentence_transformers.json (deflated 40%)
  adding: content/artifacts/sbert/2_Normalize/ (stored 0%)
  adding: content/artifacts/sbert/README.md (deflated 64%)
  adding: content/artifacts/sbert/vocab.txt (deflated 53%)
  adding: content/artifacts/sbert/modules.json (deflated 62%)
  adding: content/artifacts/sbert/tokenizer.json (deflated 71%)
  adding: content/artifacts/rnn_bilstm.keras (deflate

In [22]:
print(score_email(subject="Reminder:.",
                  body_text="   Team stand-up tomorrow 9:30 AM. Agenda: progress updates, blockers, and demo of the new feature."

                  ))

{'prob': 0.00017290146206505597, 'label': 0, 'risk': 'low', 'rnn_prob': 0.6831041574478149, 'if_anom': 0.6527727819973913}
